In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
seed = 69

In [2]:
df_finbert = pd.read_csv('finBert_sentiment.csv')
df_finbert.head()

,Unnamed: 0,title,date,stock,sentiment
0,0.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05 10:30:00-04:00,A,neutral
1,1.0,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03 10:45:00-04:00,A,neutral
2,2.0,71 Biggest Movers From Friday,2020-05-26 04:30:00-04:00,A,neutral
3,3.0,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22 12:45:00-04:00,A,neutral
4,4.0,B of A Securities Maintains Neutral on Agilent...,2020-05-22 11:38:00-04:00,A,positive


In [3]:
df_finbert.shape

(383051, 5)

## Data processing notebook
Here are the files for stock and news. Some basic processing were done to make it more concise

In [2]:
# df_stock = pd.read_csv("SPX.csv")
# df_stock.head()

In [3]:
# df_stock['Date'] = pd.to_datetime(df_stock['Date'])
# df_stock.set_index('Date', inplace=True)
# df_stock = df_stock.iloc[::-1]
# df_stock.head()

In [3]:
df_snp = pd.read_csv("all_stocks_5yr.csv")
df_snp['date'] = pd.to_datetime(df_snp['date'])
df_snp.set_index('date', inplace=True)
df_snp = df_snp.iloc[::-1]
df_snp.head()

,open,high,low,close,volume,Name
date,,,,,,
2018-02-07,72.70,75.00,72.69,73.86,4534912,ZTS
2018-02-06,72.74,74.56,72.13,73.27,4924323,ZTS
2018-02-05,76.64,76.92,73.18,73.83,2962031,ZTS
2018-02-02,77.53,78.12,76.73,76.78,2595187,ZTS
2018-02-01,76.84,78.27,76.69,77.82,2982259,ZTS


In [4]:
snp500_tickers = df_snp['Name'].unique()
snp500_tickers = snp500_tickers.tolist()

In [ ]:
# df_news = pd.read_csv("analyst_ratings_processed.csv")
# df_news.head()

In [ ]:
# df_news = df_news.dropna(subset=['title'])  # doesn't seem necessary
# del df_news['Unnamed: 0']  # this was the index
# df_news['date'] = pd.to_datetime(df_news['date'], errors='coerce', utc=True)
# df_news = df_news.dropna(subset=['date'])
# df_news.set_index('date', inplace=True)
# df_news.head()

,title,stock
date,,
2020-06-05 14:30:00+00:00,Stocks That Hit 52-Week Highs On Friday,A
2020-06-03 14:45:00+00:00,Stocks That Hit 52-Week Highs On Wednesday,A
2020-05-26 08:30:00+00:00,71 Biggest Movers From Friday,A
2020-05-22 16:45:00+00:00,46 Stocks Moving In Friday's Mid-Day Session,A
2020-05-22 15:38:00+00:00,B of A Securities Maintains Neutral on Agilent...,A


In [ ]:
# df_news = df_news[df_news['stock'].isin(snp500_tickers)]


In [ ]:
# df_news.shape, df_snp.shape

((383051, 2), (619040, 6))

In [ ]:
# df_snp.columns, df_news.columns

(Index(['open', 'high', 'low', 'close', 'volume', 'Name'], dtype='object'),
 Index(['title', 'stock'], dtype='object'))

In [3]:
sentiment_map = {'negative': -1, 'neutral': 0, 'positive': 1}
df_finbert['sentiment'] = df_finbert['sentiment'].map(sentiment_map)
df_finbert.drop(columns=['Unnamed: 0'], inplace=True)
df_finbert['date'] = pd.to_datetime(df_finbert['date'], errors='coerce', utc=True)
df_finbert.set_index('date', inplace=True)

In [4]:
df_finbert.head()

,title,stock,sentiment
date,,,
2020-06-05 14:30:00+00:00,Stocks That Hit 52-Week Highs On Friday,A,0
2020-06-03 14:45:00+00:00,Stocks That Hit 52-Week Highs On Wednesday,A,0
2020-05-26 08:30:00+00:00,71 Biggest Movers From Friday,A,0
2020-05-22 16:45:00+00:00,46 Stocks Moving In Friday's Mid-Day Session,A,0
2020-05-22 15:38:00+00:00,B of A Securities Maintains Neutral on Agilent...,A,1


In [21]:
df_finbert2 = df_finbert.groupby(['date', 'stock']).agg(
    sentiment_avg=('sentiment', 'mean'),
    sentiment_list=('sentiment', lambda s: list(s)),
    titles=('title', lambda t: list(t))
).reset_index()

In [22]:
# could be useful but we don't have enough data to justify it...
df_finbert2.head()

,date,stock,sentiment_avg,sentiment_list,titles
0,2009-04-29 12:48:00+00:00,A,0.0,[0],[Going Against the Herd]
1,2009-05-28 01:35:00+00:00,RF,0.0,[0],[MRM a $15-$20+ stock - FIT new information - ...
2,2009-06-02 02:15:00+00:00,A,0.0,[0],[super-trades - Two 52 week highs and others a...
3,2009-06-16 12:14:00+00:00,M,1.0,[1],[AIM Great News the Day After Great Earnings]
4,2009-06-20 02:44:00+00:00,ADS,0.0,[0],[Review of 3 Chinese IPO's- Week of June 22nd]


In [6]:
df_snp = pd.read_csv("stocks_with_indicators_labeled.csv")
df_snp['date'] = pd.to_datetime(df_snp['date'])
df_snp.set_index('date', inplace=True)
df_snp = df_snp.iloc[::-1]
df_snp.head()

,open,high,low,close,volume,Name,daily_variation,daily_return,sma_7,std_7,...,prev_high,prev_low,prev_close,atr,smoothed_plus_dm,smoothed_minus_dm,dx,adx,label_3,label_7
date,,,,,,,,,,,,,,,,,,,,,
2018-02-07,72.70,75.00,72.69,73.86,4534912,ZTS,0.031774,0.008052,75.112,2.044473,...,74.56,72.13,73.27,1.496558,15.160426,44.300618,49.007198,43.943384,hold,hold
2018-02-06,72.74,74.56,72.13,73.27,4924323,ZTS,0.033407,-0.007585,75.686,2.007643,...,76.92,73.18,73.83,1.433986,13.535533,53.346566,59.524197,43.164336,hold,hold
2018-02-05,76.64,76.92,73.18,73.83,2962031,ZTS,0.048800,-0.038421,76.702,1.747933,...,78.12,76.73,76.78,1.357369,16.499474,53.127271,52.605930,40.647434,hold,hold
2018-02-02,77.53,78.12,76.73,76.78,2595187,ZTS,0.017929,-0.013364,77.772,1.047459,...,78.27,76.69,77.82,1.174090,22.009727,24.352728,5.053662,38.807666,sell,hold
2018-02-01,76.84,78.27,76.69,77.82,2982259,ZTS,0.020562,0.014206,78.434,1.283211,...,78.77,76.54,76.73,1.157481,25.760238,28.502493,5.053662,44.000589,sell,hold


In [7]:
df_snp.columns

Index(['open', 'high', 'low', 'close', 'volume', 'Name', 'daily_variation',
       'daily_return', 'sma_7', 'std_7', 'ema_14', 'macd', 'macd_signal',
       'cumulative_return', 'gain', 'loss', 'rsi', 'L14', 'H14',
       'stochastic_oscillator', 'prev_high', 'prev_low', 'prev_close', 'atr',
       'smoothed_plus_dm', 'smoothed_minus_dm', 'dx', 'adx', 'label_3',
       'label_7'],
      dtype='object')

In [8]:
df_snp.rename(columns={'Name': 'stock'}, inplace=True)

df_snp.index = pd.to_datetime(df_snp.index).tz_localize('UTC').date
df_finbert.index = pd.to_datetime(df_finbert.index).date

df_snp = df_snp.reset_index().rename(columns={'index': 'date'})
df_finbert = df_finbert.reset_index().rename(columns={'index': 'date'})


In [9]:
df_finbert.columns, df_snp.columns

(Index(['date', 'title', 'stock', 'sentiment'], dtype='object'),
 Index(['date', 'open', 'high', 'low', 'close', 'volume', 'stock',
        'daily_variation', 'daily_return', 'sma_7', 'std_7', 'ema_14', 'macd',
        'macd_signal', 'cumulative_return', 'gain', 'loss', 'rsi', 'L14', 'H14',
        'stochastic_oscillator', 'prev_high', 'prev_low', 'prev_close', 'atr',
        'smoothed_plus_dm', 'smoothed_minus_dm', 'dx', 'adx', 'label_3',
        'label_7'],
       dtype='object'))

In [10]:
df_merged = df_snp.merge(
    df_finbert,
    on=['date', 'stock'],
    how='left'
)

In [11]:
df_filled = df_merged[df_merged['title'].notna()]

In [14]:
df_filled.head()

,date,open,high,low,close,volume,stock,daily_variation,daily_return,sma_7,...,prev_close,atr,smoothed_plus_dm,smoothed_minus_dm,dx,adx,label_3,label_7,title,sentiment
21,2018-01-08,73.43,74.42,73.1607,74.24,3631552,ZTS,0.017150,0.011996,72.800,...,73.36,0.912534,38.015833,11.880737,52.378540,31.919869,hold,hold,Zoetis 8-K Shows Expectation For Net Reduction...,0.0
32,2017-12-20,72.84,73.16,72.1700,72.71,1806257,ZTS,0.013591,-0.001236,72.372,...,72.80,0.937602,31.276152,15.671608,33.238100,49.148129,hold,hold,Zoetis Spikes to High of $73.08 on Volume; Act...,0.0
38,2017-12-12,71.77,72.66,71.6400,72.23,1939851,ZTS,0.014212,0.006409,71.698,...,71.77,0.944406,32.576498,7.858200,61.131401,53.054097,hold,hold,Zoetis Launches Vanguard CIV H3N2/H3N8 Vaccine...,0.0
39,2017-12-11,71.84,71.97,71.5200,71.77,1172513,ZTS,0.006264,-0.001530,71.422,...,71.88,0.938591,26.511224,9.123329,48.795041,51.811435,hold,hold,Zoetis Raises Qtr. Dividend From $.105/Share T...,1.0
47,2017-11-29,71.99,72.10,71.2800,71.61,1618690,ZTS,0.011390,0.003082,71.380,...,71.39,0.925980,34.266185,4.607591,76.294604,60.906298,hold,hold,Analyst: Zoetis Deserves To Trade At A Premium,0.0


In [13]:
stock_counts = df_filled['stock'].value_counts()
print(stock_counts.describe())


count     392.000000
mean      412.696429
std       348.523091
min         1.000000
25%       179.000000
50%       297.000000
75%       495.000000
max      1984.000000
Name: count, dtype: float64


In [15]:
df_filled.to_csv('combined_data.csv', index=False)


In [18]:
from sklearn.model_selection import train_test_split

In [19]:
train_val_df, test_df = train_test_split(df_filled, test_size=0.15, random_state=42, shuffle=True)
train_df, val_df = train_test_split(train_val_df, test_size=0.1765, random_state=42, shuffle=True)  # 0.1765 * 0.85 = 0.15
print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

Train: 113239, Val: 24271, Test: 24267


In [20]:
train_df.to_csv('training_data.csv', index=False)
val_df.to_csv('validation_data.csv', index=False)
test_df.to_csv('testing_data.csv', index=False)

In [21]:
test_df.columns

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'stock',
       'daily_variation', 'daily_return', 'sma_7', 'std_7', 'ema_14', 'macd',
       'macd_signal', 'cumulative_return', 'gain', 'loss', 'rsi', 'L14', 'H14',
       'stochastic_oscillator', 'prev_high', 'prev_low', 'prev_close', 'atr',
       'smoothed_plus_dm', 'smoothed_minus_dm', 'dx', 'adx', 'label_3',
       'label_7', 'title', 'sentiment'],
      dtype='object')

In [22]:
train_df['label_3'].value_counts()

label_3
hold    85371
buy     15021
sell    12847
Name: count, dtype: int64

In [24]:
train_df['label_7'].value_counts()

label_7
hold    75547
buy     20855
sell    16837
Name: count, dtype: int64

In [30]:
train_df.iloc[1].tolist()

[datetime.date(2015, 3, 2),
 211.96,
 217.0,
 211.96,
 215.0,
 732621,
 'WHR',
 0.0237780713342139,
 0.0143901863647086,
 213.64,
 1.2356172546792172,
 212.8283613438811,
 -0.0002575667142407,
 -0.0005904783303516,
 100.33544539694373,
 0.0143901863647086,
 0.0,
 76.6809156016469,
 205.33,
 217.0,
 82.86203941730932,
 213.96,
 211.254,
 211.95,
 4.0170701728355525,
 23.95644322958968,
 9.195249497337866,
 44.5262142534942,
 36.58629173341652,
 'hold',
 'sell',
 'Top Performing Industries For March 2, 2015',
 0.0]